In [1]:
#!/bin/python
import json
import os
from io import StringIO

import numpy as np
import pandas as pd

import dimod
import hybrid

import networkx as nx
from dimod import SampleSet
from hybrid import KerberosSampler

### Path to protein pdb file

In [2]:
PDB_FILE = "inputs/6a5j.pdb"

### Execute the [InteractionGraphSummaryMetric](https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/SimpleMetrics/simple_metric_pages/InteractionGraphSummaryMetric) from RosettaScripts specified in file `rscript/script.xml` 

In [ ]:
os.system(f"rscript/rosetta_scripts -parser:protocol rscript/script.xml -scorefile_format json -s {PDB_FILE}")

core.init: Checking for fconfig files in pwd and ./rosetta/flags 
core.init: Rosetta version: rosetta.binary.linux.release-280 r280 2021.16+release.8ee4f02 8ee4f02ac5768a8a339ffada74cb0ff5f778b3e6 https://www.rosettacommons.org 2021-04-20T20:52:25.363712
core.init: command: rscript/rosetta_scripts -parser:protocol rscript/script.xml -scorefile_format json -s inputs/6a5j.pdb
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=2119094205 seed_offset=0 real_seed=2119094205
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=2119094205 RG_type=mt19937
core.init: Resolved executable path: /home/sewerin/workspace/magisterka/rosetta/rosetta_bin_linux_2021.16.61629_bundle/main/source/build/src/release/linux/3.10/64/x86/gcc/4.8/static/rosetta_scripts.static.linuxgccrelease
core.init: Looking for database based on location of executable: /home/sewerin/workspace/magisterka/rosetta/rosetta_bin_linux_2021.16.61629_bundle/main/database/
prot

### The script executed above outputs to a file named `out.txt`
It consists of 4 parts:
 1. binary information about the pose,
 2. information about the possible rotamers:
    - residue name,
    - position in the pose,
    - rotamer index for this position,
    - "side-chain chi angle values for each rotatable dihedral",
    - additional binary information with the detailed three-dimensional geometry,
 3. one-body energies
 4. two-body energies
 
For the purpose of constructing the QUBO only the last two parts are used

### Constants used to find parts 3 and 4 of the output file

In [2]:
ONEBODY_BEGIN_TAG = '[BEGIN ONEBODY SEQPOS/ROTINDEX/ENERGY]'
ONEBODY_END_TAG = '[END ONEBODY SEQPOS/ROTINDEX/ENERGY]'
TWOBODY_BEGIN_TAG = '[BEGIN TWOBODY SEQPOS1/ROTINDEX1/SEQPOS2/ROTINDEX2/ENERGY]'
TWOBODY_END_TAG = '[END TWOBODY SEQPOS1/ROTINDEX1/SEQPOS2/ROTINDEX2/ENERGY]'

### Parsing the data into pandas dataframes

In [ ]:
with open('out.txt', 'r') as f:
    file_text = f.read()

onebody_begin = file_text.find(ONEBODY_BEGIN_TAG) + len(ONEBODY_BEGIN_TAG) + 1
onebody_end = file_text.find(ONEBODY_END_TAG, onebody_begin)
twobody_begin = file_text.find(TWOBODY_BEGIN_TAG, onebody_end) + len(TWOBODY_BEGIN_TAG) + 1
twobody_end = len(file_text) - len(TWOBODY_END_TAG) - 2

onebody_text = file_text[onebody_begin:onebody_end]
twobody_text = file_text[twobody_begin:twobody_end]

onebody_energies = pd.read_csv(StringIO(onebody_text), sep='\t', names=['position', 'rotamer_index', 'energy'])
nodes = list(zip(onebody_energies['position'], onebody_energies['rotamer_index']))
onebody_energies['pr'] = nodes
twobody_energies = pd.read_csv(StringIO(twobody_text), sep='\t', names=['position_1', 'rotamer_index_1', 'position_2', 'rotamer_index_2', 'energy'])
twobody_energies['pr1'] = list(zip(twobody_energies['position_1'], twobody_energies['rotamer_index_1']))
twobody_energies['pr2'] = list(zip(twobody_energies['position_2'], twobody_energies['rotamer_index_2']))

### One option in the API to create a QUBO is from an adjacency matrix. The `twobody_energies` dataframe is an edge list of energies between two rotameter pairs. To quickly convert between the two graph representations we are using networkx. 

In [3]:
graph = nx.from_pandas_edgelist(twobody_energies, 'pr1', 'pr2', 'energy')
qubo_np = nx.to_numpy_matrix(graph, nodelist=nodes, weight='energy')

### `qubo_np` will contain the final matrix representation of the QUBO.  
One-body energies are added on the main diagonal.  
Also it's required to ensure that no two different rotamers are selected for a single position. In order to constrain that a big energy value is placed between any two qubits representing rotamers on the same position.

In [3]:
big_energy = max(onebody_energies['energy'].max(), twobody_energies['energy'].max()) * 1e9

N = len(onebody_energies)

# qubo_np = np.zeros((N, N))

# Mapping from qubit index to pair (position, index) identifying the rotamer and vice versa
idx_to_pr = {index: row['pr'] for index, row in onebody_energies.iterrows()}
pr_to_idx = {row['pr']: index for index, row in onebody_energies.iterrows()}

for index_1, row_1 in onebody_energies.iterrows():
    pos_1, rot_1 = idx_to_pr[index_1]
    qubo_np[index_1, index_1] = row_1['energy']
    for index_2 in range(index_1 + 1, N):
        pos_2, rot_2 = idx_to_pr[index_2]
        if pos_1 == pos_2:
            qubo_np[index_1, index_2] = big_energy

### Create the BQM (Binary Quadratic Model)

In [4]:
# bqm = dimod.BinaryQuadraticModel.from_numpy_matrix(qubo_np)
bqm = dimod.BinaryQuadraticModel(qubo_np, 'BINARY')

### The qubo is too big to be simply executed on current quantum computers. A hybrid approach is required.
### Using the reference [Kerberos Sampler](https://docs.ocean.dwavesys.com/projects/hybrid/en/latest/intro/using.html#reference-hybrid-sampler-kerberos) to produce results

In [5]:
print("--- SAMPLING ---")
solution: SampleSet = KerberosSampler().sample(bqm, max_iter=10, convergence=3)

--- SAMPLING ---


In [6]:
solution

SampleSet(rec.array([([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# Save solution for later use
s = json.dumps(solution.to_serializable())
with open("samples.out.json", "w") as out:
    out.write(s)

### Because of the nature of the problem, for many positions a rotamer wasn't selected
### To add additional constraints the BQM is converted to CQM (Constrained Quadratic Model)

In [13]:
cqm = dimod.ConstrainedQuadraticModel.from_quadratic_model(bqm)

In [19]:
mapping = {i: f"{i}" for i in range(N)}

cqm.relabel_variables(mapping)

### Create a list of sets of qubits representing a rotamer on the same position

In [41]:
constraints = []
constraint = []
k = 1
for idx, pr in idx_to_pr.items():
    if pr[0] != k:
        constraints.append(constraint)
        constraint = []
        k = pr[0]
    constraint.append(f"{idx}")
constraints.append(constraint)

### For every position an additional constraint is added - the sum of qubits representing rotamers on this position must be equal to one

In [32]:
for constraint in constraints:
    c = map(lambda x: (x, 1), constraint)
    cqm.add_constraint_from_iterable(c, '==', rhs=1)

In [35]:
print("--- SAMPLING CQM ---")

bqm2, invert = dimod.cqm_to_bqm(cqm)
solutionCQM: SampleSet = KerberosSampler().sample(bqm2, max_iter=10, convergence=3)

--- SAMPLING CQM ---


### The solution is a list of all rotamers with the information whether it was selected or not. Using the (position, index) pair additional information can be extracted from the output file of the Rosetta script, or using [ExternalPackerResultLoader](https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/ExternalPackerResultLoader) the results can be loaded back into Rosetta Scripts to use it in the identical way to the standard Rosetta Packer results.  

In [39]:
inverted = invert(solutionCQM.first.sample)
for idx, pr in idx_to_pr.items():
    print(idx, pr, inverted[f"{idx}"], sep="\t")

0	(1, 1)	0
1	(1, 2)	0
2	(1, 3)	0
3	(1, 4)	0
4	(1, 5)	0
5	(1, 6)	0
6	(1, 7)	0
7	(1, 8)	0
8	(1, 9)	0
9	(1, 10)	1
10	(1, 11)	0
11	(2, 1)	1
12	(2, 2)	0
13	(2, 3)	0
14	(2, 4)	0
15	(2, 5)	0
16	(2, 6)	0
17	(2, 7)	0
18	(2, 8)	0
19	(3, 1)	1
20	(3, 2)	0
21	(3, 3)	0
22	(3, 4)	0
23	(3, 5)	0
24	(3, 6)	0
25	(3, 7)	0
26	(3, 8)	0
27	(4, 1)	1
28	(4, 2)	0
29	(4, 3)	0
30	(4, 4)	0
31	(4, 5)	0
32	(4, 6)	0
33	(4, 7)	0
34	(4, 8)	0
35	(5, 1)	1
36	(5, 2)	0
37	(5, 3)	0
38	(5, 4)	0
39	(5, 5)	0
40	(5, 6)	0
41	(5, 7)	0
42	(5, 8)	0
43	(6, 1)	1
44	(6, 2)	0
45	(6, 3)	0
46	(6, 4)	0
47	(6, 5)	0
48	(6, 6)	0
49	(6, 7)	0
50	(6, 8)	0
51	(7, 1)	1
52	(7, 2)	0
53	(7, 3)	0
54	(7, 4)	0
55	(7, 5)	0
56	(7, 6)	0
57	(7, 7)	0
58	(7, 8)	0
59	(8, 1)	1
60	(8, 2)	0
61	(8, 3)	0
62	(8, 4)	0
63	(8, 5)	0
64	(8, 6)	0
65	(9, 1)	1
66	(9, 2)	0
67	(9, 3)	0
68	(9, 4)	0
69	(9, 5)	0
70	(9, 6)	0
71	(9, 7)	0
72	(9, 8)	0
73	(10, 1)	1
74	(10, 2)	0
75	(10, 3)	0
76	(10, 4)	0
77	(10, 5)	0
78	(10, 6)	0
79	(10, 7)	0
80	(10, 8)	0
81	(11, 1)	1
82	(11, 2)	0
83